In [2]:
import numpy as np
import mne
import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
import plotly.io as pio
import plotly.subplots as sp
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import pandas as pd
import os

# Configurar Plotly para usar o renderer 'browser'
pio.renderers.default = 'browser'

# Diretório absoluto onde os arquivos serão salvos
project_dir = os.path.expanduser('~/arquivoscode/projetoEGG')
output_dir_base = os.path.join(project_dir, 'dados_eeg')
os.makedirs(output_dir_base, exist_ok=True)

# Função para exibir a imagem do sistema 10-20
def exibir_imagem_10_20():
    img = plt.imread('Electrode-montage-We-used-a-standard-extension-of-the-international-10-20-system-to - Copia.png')
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Sistema 10-20 - Selecione os eletrodos:')
    plt.show()

# Função para obter os eletrodos selecionados
def obter_eletrodos_selecionados():
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

# Função para determinar a melhor combinação de tipos de ondas para os eletrodos selecionados
def definir_tipos_onda(eletrodos_selecionados):
    tipos_onda = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    return {eletrodo: tipos_onda[i % len(tipos_onda)] for i, eletrodo in enumerate(eletrodos_selecionados)}

# Criar checkboxes para cada eletrodo (excluindo A1 e A2)
eletrodos_disponiveis = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FC5", "FC1", "FC2", "FC6", "T7", "C3", "Cz", "C4", "T8", "CP5", "CP1", "CP2", "CP6", "P7", "P3", "Pz", "P4", "P8", "PO7", "PO3", "POz", "PO4", "PO8", "O1", "Oz", "O2"]
checkboxes = [widgets.Checkbox(value=False, description=eletrodo) for eletrodo in eletrodos_disponiveis]

# Exibir a imagem do sistema 10-20
exibir_imagem_10_20()

# Exibir os checkboxes
for checkbox in checkboxes:
    display(checkbox)

# Ajustar widgets para frequência de amostragem e tempo de simulação
frequencia_amostragem_widget = widgets.FloatText(value=256.0, description='Frequência de Amostragem:', layout=widgets.Layout(width='300px'))
tempo_simulacao_widget = widgets.FloatText(value=10.0, description='Tempo de Simulação:', layout=widgets.Layout(width='300px'))

# Organizar os widgets em um VBox
input_box = widgets.VBox([frequencia_amostragem_widget, tempo_simulacao_widget])
display(input_box)

# Botão para iniciar a definição dos parâmetros
botao_iniciar = widgets.Button(description="Iniciar")
display(botao_iniciar)

# Função para gerar sinais de EEG
def gerar_sinais_eeg(parametros):
    try:
        n_channels = len(parametros['eletrodos'])
        frequencia_amostragem = int(parametros['frequencia_amostragem'])
        tempo_simulacao = int(parametros['tempo_simulacao'])
        n_times = frequencia_amostragem * tempo_simulacao
        data = np.zeros((n_channels, n_times))
        info = mne.create_info(parametros['eletrodos'], frequencia_amostragem, ch_types='eeg')
        raw = mne.io.RawArray(data, info)
        
        caracteristicas_ondas = {
            'delta': {'amplitude': 20, 'frequencia': 2, 'desvio_padrao': 0.5},
            'theta': {'amplitude': 10, 'frequencia': 6, 'desvio_padrao': 1},
            'alpha': {'amplitude': 15, 'frequencia': 10, 'desvio_padrao': 1.5},
            'beta': {'amplitude': 5, 'frequencia': 20, 'desvio_padrao': 2},
            'gamma': {'amplitude': 3, 'frequencia': 40, 'desvio_padrao': 3}
        }

        t = np.linspace(0, parametros['tempo_simulacao'], n_times, endpoint=False)

        for eletrodo, tipo_onda in parametros['tipos_onda'].items():
            amplitude = caracteristicas_ondas[tipo_onda]['amplitude']
            frequencia = caracteristicas_ondas[tipo_onda]['frequencia']
            desvio_padrao = caracteristicas_ondas[tipo_onda]['desvio_padrao']

            onda_simulada = amplitude * np.sin(2 * np.pi * frequencia * t + np.random.normal(scale=desvio_padrao, size=t.shape))
            idx_eletrodo = parametros['eletrodos'].index(eletrodo)
            raw._data[idx_eletrodo, :] += onda_simulada

        ruido_amplitude = 5
        noise = np.random.normal(scale=ruido_amplitude, size=raw._data.shape)
        raw._data += noise

        for idx in range(n_channels):
            if np.random.rand() < 0.2:
                batimento_frequencia = np.random.uniform(1, 2)
                batimento = 50 * np.sin(2 * np.pi * batimento_frequencia * t)
                raw._data[idx, :] += batimento

            if np.random.rand() < 0.1:
                piscada_frequencia = np.random.uniform(0.5, 1)
                piscada = 100 * np.sin(2 * np.pi * piscada_frequencia * t)
                raw._data[idx, :] += piscada

        raw.notch_filter(freqs=50, notch_widths=1, method='spectrum_fit', filter_length='auto', phase='zero')
        raw.filter(None, 40., fir_design='firwin')

        return raw, t
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

# Função para plotar sinais de EEG
def plotar_sinais_eeg(raw, t, parametros):
    n_channels = len(parametros['eletrodos'])
    figs = []
    for page in range(0, n_channels, 5):
        fig = sp.make_subplots(rows=min(5, n_channels - page), cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=parametros['eletrodos'][page:page+5])
        for idx in range(page, min(page + 5, n_channels)):
            fig.add_trace(
                go.Scatter(x=t, y=raw._data[idx, :], mode='lines', name=parametros['eletrodos'][idx]),
                row=idx - page + 1, col=1
            )
        fig.update_layout(height=600 * min(5, n_channels - page), title_text=f"Sinais de EEG Simulados - Página {page//5 + 1}", showlegend=False)
        fig.show()
        figs.append(fig)
    return figs

# Função para plotar espectrogramas
def plotar_spectrogramas(raw, parametros):
    n_channels = len(parametros['eletrodos'])
    figs = []
    for page in range(0, n_channels, 5):
        fig_spectrogram = sp.make_subplots(rows=min(5, n_channels - page), cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=parametros['eletrodos'][page:page+5])
        for idx in range(page, min(page + 5, n_channels)):
            f, t_spec, Sxx = spectrogram(raw._data[idx], fs=parametros['frequencia_amostragem'])
            fig_spectrogram.add_trace(
                go.Heatmap(z=10 * np.log10(Sxx), x=t_spec, y=f, colorscale='Viridis', showscale=(idx == page)),
                row=idx - page + 1, col=1
            )
        fig_spectrogram.update_layout(height=600 * min(5, n_channels - page), title_text=f"Spectrogramas de EEG Simulados - Página {page//5 + 1}", showlegend=False, margin=dict(r=20))
        fig_spectrogram.update_yaxes(title_text='Frequência (Hz)', row=1, col=1)
        fig_spectrogram.update_xaxes(title_text='Tempo (s)', row=min(5, n_channels - page), col=1)
        fig_spectrogram.show()
        figs.append(fig_spectrogram)
    return figs

# Função para gerar estatísticas
def gerar_estatisticas(raw, parametros):
    estatisticas = {}
    for idx, eletrodo in enumerate(parametros['eletrodos']):
        data = raw._data[idx]
        estatisticas[eletrodo] = {
            'Média': np.mean(data),
            'Mediana': np.median(data),
            'Desvio Padrão': np.std(data),
            'Mínimo': np.min(data),
            'Máximo': np.max(data)
        }
    return estatisticas

# Função para plotar histogramas de amplitude dos sinais
def plotar_histogramas(raw, parametros):
    n_channels = len(parametros['eletrodos'])
    figs = []
    for page in range(0, n_channels, 5):
        fig_hist = sp.make_subplots(rows=min(5, n_channels - page), cols=1, subplot_titles=parametros['eletrodos'][page:page+5])
        for idx in range(page, min(page + 5, n_channels)):
            fig_hist.add_trace(
                go.Histogram(x=raw._data[idx, :], nbinsx=50, name=parametros['eletrodos'][idx]),
                row=idx - page + 1, col=1
            )
        fig_hist.update_layout(height=600 * min(5, n_channels - page), title_text=f"Histogramas de Amplitude dos Sinais - Página {page//5 + 1}", showlegend=False)
        fig_hist.show()
        figs.append(fig_hist)
    return figs

# Função chamada ao clicar no botão
def on_parametros_confirmados(b):
    clear_output()
    output_dir = os.path.join(output_dir_base, f'dados_eeg_pessoa_{len(os.listdir(output_dir_base)) + 1}')
    os.makedirs(output_dir, exist_ok=True)
    
    parametros = {
        'frequencia_amostragem': frequencia_amostragem_widget.value,
        'tempo_simulacao': tempo_simulacao_widget.value,
        'eletrodos': obter_eletrodos_selecionados(),
        'tipos_onda': definir_tipos_onda(obter_eletrodos_selecionados())
    }

    raw, t = gerar_sinais_eeg(parametros)
    figs_sinais = plotar_sinais_eeg(raw, t, parametros)
    figs_spectrogramas = plotar_spectrogramas(raw, parametros)
    figs_histogramas = plotar_histogramas(raw, parametros)
    
    estatisticas = gerar_estatisticas(raw, parametros)
    for eletrodo, stats in estatisticas.items():
        print(f"Estatísticas para o eletrodo {eletrodo}:")
        for key, value in stats.items():
            print(f"  {key}: {value}")
        print()
    
    # Salvar os dados em arquivos CSV
    salvar_sinais_csv(raw, parametros, output_dir)
    salvar_fft_csv(raw, parametros, output_dir)
    salvar_histogramas_csv(raw, parametros, output_dir)
    salvar_estatisticas_csv(estatisticas, output_dir)
    salvar_parametros_json(parametros, output_dir)

    # Perguntar se deseja salvar os gráficos
    display(widgets.HTML("<h3>Deseja salvar os gráficos gerados?</h3>"))
    botao_salvar = widgets.Button(description="Salvar Gráficos")
    display(botao_salvar)
    
    def on_salvar_graficos(b):
        # Salvar os gráficos como arquivos HTML interativos
        sinais_dir = os.path.join(output_dir, 'sinais')
        spectrogramas_dir = os.path.join(output_dir, 'spectrogramas')
        histogramas_dir = os.path.join(output_dir, 'histogramas')
        os.makedirs(sinais_dir, exist_ok=True)
        os.makedirs(spectrogramas_dir, exist_ok=True)
        os.makedirs(histogramas_dir, exist_ok=True)

        for i, fig in enumerate(figs_sinais):
            sinais_path = os.path.join(sinais_dir, f'sinais_eeg_pagina_{i + 1}.html')
            fig.write_html(sinais_path)
        for i, fig in enumerate(figs_spectrogramas):
            spectrogramas_path = os.path.join(spectrogramas_dir, f'spectrogramas_eeg_pagina_{i + 1}.html')
            fig.write_html(spectrogramas_path)
        for i, fig in enumerate(figs_histogramas):
            histogramas_path = os.path.join(histogramas_dir, f'histogramas_eeg_pagina_{i + 1}.html')
            fig.write_html(histogramas_path)

        print(f"Gráficos salvos em HTML no diretório: {output_dir}")
    
    botao_salvar.on_click(on_salvar_graficos)

# Função para salvar sinais em arquivo CSV
def salvar_sinais_csv(raw, parametros, output_dir):
    n_channels = len(parametros['eletrodos'])
    data = raw.get_data()
    df = pd.DataFrame(data.T, columns=parametros['eletrodos'])
    file_path = os.path.join(output_dir, 'sinais_eeg.csv')
    df.to_csv(file_path, index=False)
    print(f"Arquivo salvo em: {file_path}")

# Função para calcular e salvar FFT em arquivo CSV
def salvar_fft_csv(raw, parametros, output_dir):
    n_channels = len(parametros['eletrodos'])
    fft_data = np.fft.fft(raw.get_data(), axis=1)
    freqs = np.fft.fftfreq(raw.n_times, 1/parametros['frequencia_amostragem'])
    df = pd.DataFrame(fft_data.T, columns=parametros['eletrodos'])
    df['Frequência'] = freqs
    file_path = os.path.join(output_dir, 'fft_eeg.csv')
    df.to_csv(file_path, index=False)
    print(f"Arquivo salvo em: {file_path}")

# Função para salvar histogramas em arquivo CSV
def salvar_histogramas_csv(raw, parametros, output_dir):
    n_channels = len(parametros['eletrodos'])
    data = raw.get_data()
    histogram_data = {}
    for idx, eletrodo in enumerate(parametros['eletrodos']):
        hist, bin_edges = np.histogram(data[idx, :], bins=50)
        histogram_data[f'{eletrodo}_hist'] = hist
        histogram_data[f'{eletrodo}_bins'] = bin_edges[:-1]
    df = pd.DataFrame(histogram_data)
    file_path = os.path.join(output_dir, 'histogramas_eeg.csv')
    df.to_csv(file_path, index=False)
    print(f"Arquivo salvo em: {file_path}")

# Função para salvar estatísticas em arquivo CSV
def salvar_estatisticas_csv(estatisticas, output_dir):
    df = pd.DataFrame(estatisticas).T
    file_path = os.path.join(output_dir, 'estatisticas_eeg.csv')
    df.to_csv(file_path, index=True)
    print(f"Arquivo salvo em: {file_path}")

# Função para salvar parâmetros em JSON
def salvar_parametros_json(parametros, output_dir):
    file_path = os.path.join(output_dir, 'parametros_simulacao.json')
    with open(file_path, 'w') as f:
        json.dump(parametros, f)
    print(f"Arquivo salvo em: {file_path}")

# Botão para confirmar parâmetros e gerar sinais
botao_iniciar.on_click(on_parametros_confirmados)



Creating RawArray with float64 data, n_channels=11, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     9.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Removed notch frequencies (Hz):
     50.00 :   11 windows
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)


- Filter length: 85 samples (0.332 s)

Estatísticas para o eletrodo Fp1:
  Média: 0.23012855566105053
  Mediana: 0.45116168583358796
  Desvio Padrão: 13.483333329433574
  Mínimo: -27.198346357320048
  Máximo: 25.8843417865929

Estatísticas para o eletrodo Fp2:
  Média: -0.16844917490194752
  Mediana: 0.7751840491860449
  Desvio Padrão: 36.194589518034924
  Mínimo: -65.49167287968201
  Máximo: 62.36346539233751

Estatísticas para o eletrodo F7:
  Média: 0.018577458507287297
  Mediana: -0.1223773472666202
  Desvio Padrão: 7.670343962141532
  Mínimo: -24.40144007229236
  Máximo: 21.194980912116087

Estatísticas para o eletrodo F3:
  Média: 0.1563164902169177
  Mediana: -3.614752661297663
  Desvio Padrão: 78.17813061539218
  Mínimo: -153.88897270782223
  Máximo: 149.67517000054426

Estatísticas para o eletrodo Fz:
  Média: 0.1128871225795115
  Mediana: 0.11197204125651106
  Desvio Padrão: 3.075827572284011
  Mínimo: -11.828072834094595
  Máximo: 11.476665320204509

Estatísticas para o elet

HTML(value='<h3>Deseja salvar os gráficos gerados?</h3>')

Button(description='Salvar Gráficos', style=ButtonStyle())

Gráficos salvos em HTML no diretório: C:\Users\renan/arquivoscode/projetoEGG\dados_eeg\dados_eeg_pessoa_1
